In [5]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display

In [6]:
df = pd.read_csv('data/zamowienia.csv', sep=';')
df.head()

,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,Polska,Kowalski,2003-07-16,10248,440.00
1,Polska,Sowiński,2003-07-10,10249,1863.40
2,Niemcy,Peacock,2003-07-12,10250,1552.60
3,Niemcy,Leverling,2003-07-15,10251,654.06
4,Niemcy,Peacock,2003-07-11,10252,3597.90


In [7]:
df['Data zamowienia'] = pd.to_datetime(df['Data zamowienia'])

In [8]:
year = widgets.Dropdown(
    description='Rok:',
    value=np.min(df['Data zamowienia'].dt.year),
    options=df['Data zamowienia'].dt.year.unique().tolist()
)
seller = widgets.Dropdown(
    description='Sprzedawca:',
    value=df['Sprzedawca'].unique()[0],
    options=df['Sprzedawca'].unique().tolist()
)
country = widgets.Dropdown(
    description='Kraj:',
    value=df['Kraj'].unique()[0],
    options=df['Kraj'].unique().tolist()
)
agg = widgets.Dropdown(
    description='Agregacja:',
    value='Suma',
    options=['Suma', 'Średnia']
)

In [9]:
trace1 = go.Histogram(x=df['Utarg'], opacity=0.75, name='Utarg')
trace2 = go.Histogram(x=df['Sprzedawca'], opacity=0.75, name='Sprzedawcy')
graph = go.FigureWidget(data=[trace1, trace2], layout=go.Layout(
    title=dict(
        text='Zamówienia'
    ),
    barmode='overlay'
))

In [10]:
def response(change):
    filter_list = [i and j and k for i, j, k in
        zip(df['Data zamowienia'].dt.year == year.value,
            df['Sprzedawca'] == seller.value,
            df['Kraj'] == country.value)]
    temp_df = df[filter_list]
    if agg.value == 'Suma':
        x1 = temp_df['Utarg'].sum()
    else:
        x1 = temp_df['Utarg'].mean()
    x2 = temp_df['Sprzedawca']
    with graph.batch_update():
        graph.data[0].x = x1
        graph.data[1].x = x2
        graph.layout.barmode = 'overlay'
        graph.layout.xaxis.title = 'Utarg'
        graph.layout.yaxis.title = 'Sprzedawcy'


year.observe(response, names='value')
seller.observe(response, names='value')
country.observe(response, names='value')
agg.observe(response, names='value')

In [11]:
container = widgets.HBox([year, seller, country, agg])
w = widgets.VBox([container,
              graph])
display(w)